# Consultas propuestas por el enunciado

#### Imports y definición de funciones

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

import re

import warnings
# suprimir future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
spark = SparkSession.builder\
    .master("local[*]") \
    .appName("ConsultasEnunciado") \
    .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/26 16:15:48 WARN Utils: Your hostname, LENOVOID-ubuntu, resolves to a loopback address: 127.0.1.1; using 192.168.10.209 instead (on interface wlp2s0)
25/09/26 16:15:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/26 16:15:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1. ¿Cuál es el Estado que más descuentos tiene en total? ¿y en promedio?

#### Hipótesis
- Los estados con más descuentos son aquellos que tienen más órdenes registradas con un `discount_amount` no nulo y mayor a ceros.
- Se tienen en cuenta los estados de la columna `billing_address`.
- Se considera que el estado está definido por un conjunto de dos letras mayúsculas seguido de un espacio y cinco dígitos (código postal).
- No se consideran los estados que no están definidos por el anterior patrón, ni tampoco las filas con un valor nulo en `billing_address`.

#### Limpieza de datos para la tabla de órdenes

In [ ]:
def state_and_zip_code(address):
    if address is None:
        return "UNDEFINED", None
    pattern = r'([A-Z]{2})\s(\d{5})'
    match = re.search(pattern, address)
    if match:
        return match.group(1), int(match.group(2))
    return "UNDEFINED", None

def retain_orders_columns(row: Row):
    state, zip_code = state_and_zip_code(row.billing_address)
    status = "UNDEFINED" if row.status is None else row.status.strip().upper()
    return (
        row.customer_id,
        row.discount_amount,
        status,
        state,
        zip_code,
    )
    
ordersIdx = {
    "customer_id": 0,
    "discount_amount": 1,
    "status": 2,
    "state": 3,
    "zip_code": 4,
}

In [ ]:
orders = sqlContext.read.csv(
    'data/orders.csv',
    header=True, inferSchema=True
)
ordersRDD = orders.rdd.map(retain_orders_columns).persist()
ordersRDD.first()

25/09/26 16:16:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, customer_id, order_date, status, payment_method, shipping_address, billing_address, discount_amount, tax_amount, shipping_cost, total_amount, currency, created_at, updated_at, subtotal
 Schema: _c0, order_id, customer_id, order_date, status, payment_method, shipping_address, billing_address, discount_amount, tax_amount, shipping_cost, total_amount, currency, created_at, updated_at, subtotal
Expected: _c0 but found: 
CSV file: file:///home/pat/Documents/GitHub/datos-tp2/data/orders.csv


(447917, 0.0, 'COMPLETED', 'AP', 90901)

#### Resolución

In [5]:
discountAndTotalOrdersByState = ordersRDD \
    .filter(lambda x: x[ordersIdx["state"]] != "UNDEFINED") \
    .map(
        lambda row: (
            row[ordersIdx["state"]], 
            (1 if (row[ordersIdx["discount_amount"]] is not None and row[ordersIdx["discount_amount"]] > 0) else 0, 1)
        )) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .cache()
# (estado, (#ordenes con descuentos, #ordenes totales))

In [6]:
highestCount = discountAndTotalOrdersByState\
    .reduce(lambda a, b: a if a[1][0] > b[1][0] else b)

highestAvg = discountAndTotalOrdersByState\
    .reduce(lambda a, b: a if a[1][0]/a[1][1] > b[1][0]/b[1][1] else b)

In [ ]:
print(f"El Estado con más órdenes con descuentos es {highestCount[0]} con {highestCount[1][0]} órdenes con descuentos.")

print(f"\nEl Estado con el mayor promedio de descuentos es {highestAvg[0]} con un promedio de {highestAvg[1][0]/highestAvg[1][1]:.2f}.")

El Estado con más órdenes con descuentos es AE con 30858 órdenes con descuentos.

El Estado con el mayor promedio de descuentos es KY con un promedio de 0.22.


### 2. ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado 'Refunded'? 
###    ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

#### Limpieza de datos para la tabla de usuarios

In [18]:
def retain_customers_columns(row: Row):
    first_name = "UNDEFINED" if row.first_name is None else row.first_name.strip().upper()
    segment = "UNDEFINED" if row.customer_segment is None else row.customer_segment.strip().upper()
    is_active = False if row.is_active is None else row.is_active
    marketing_consent = False if row.marketing_consent is None else row.marketing_consent
    return (
        row.customer_id,
        first_name,
        row.postal_code,
        segment,
        is_active,
        marketing_consent,
    )
    
customersIdx = {
    "id": 0,
    "name": 1,
    "zip_code": 2,
    "segment": 3,
    "is_active": 4,
    "consent": 5,
}

In [19]:
customers = sqlContext.read.csv(
    'data/customers.csv',
    header=True, inferSchema=True
)
customersRDD = customers.rdd.map(retain_customers_columns).persist()
customersRDD.first()

25/09/26 16:53:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, email, first_name, last_name, phone, date_of_birth, gender, country, city, postal_code, address, registration_date, last_login, is_active, customer_segment, marketing_consent
 Schema: _c0, customer_id, email, first_name, last_name, phone, date_of_birth, gender, country, city, postal_code, address, registration_date, last_login, is_active, customer_segment, marketing_consent
Expected: _c0 but found: 
CSV file: file:///home/pat/Documents/GitHub/datos-tp2/data/customers.csv


(1, 'KAYLA', 70351, 'REGULAR', True, True)

#### Resolución

In [20]:
zipCodesWithMostRefundedOrdersCount = ordersRDD \
    .filter(
        lambda row: (
            row[ordersIdx["zip_code"]] is not None
            and row[ordersIdx["status"]] == "REFUNDED"
        )
    ) \
    .map(lambda row: (row[ordersIdx["zip_code"]], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(5, key=lambda x: -x[1])

topRefundedZipCodes = [zip_code for zip_code, _ in zipCodesWithMostRefundedOrdersCount]

In [30]:
mostFrecuentNameInTopRefundedZipCodes = customersRDD \
    .filter(
        lambda row: (
            row[customersIdx["zip_code"]] in topRefundedZipCodes
            and row[customersIdx["name"]] != "UNDEFINED"
        )
    ) \
    .map(lambda row: (row[customersIdx["name"]], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .reduce(lambda a, b: a if a[1] > b[1] else b)

In [31]:
print("Los 5 códigos postales con más órdenes reembolsadas son:")
print("\tCodigo\tÓrdenes reembolsadas")
for zip_code, count in zipCodesWithMostRefundedOrdersCount:
    print(f"\t{zip_code}\t{count}")

print(f"\nEl nombre más frecuente entre los clientes de esos códigos postales es: {mostFrecuentNameInTopRefundedZipCodes[0]}")

Los 5 códigos postales con más órdenes reembolsadas son:
	Codigo	Órdenes reembolsadas
	31571	6
	14396	5
	9045	5
	38151	5
	91623	5

El nombre más frecuente entre los clientes de esos códigos postales es: MICHAEL


### 3. Para cada tipo de pago y segmento de cliente, <br>devolver la suma y el promedio expresado como porcentaje <br>de clientes activos y de consentimiento de marketing

#### Resolución

### 4. Para los productos que contienen en su descripción la palabra "stuff", <br> calcular el peso total de su inventario agrupado por marca, <br> mostrar sólo la marca y el peso total de las 5 más pesadas

#### Limpieza y preparación de columnas

#### Resolución

### 5. Calcular el porcentaje de productos cuyo stock es al menos 20% <br> más alto que el stock promedio de su marca

#### Limpieza y preparación de columnas

#### Resolución

### 6. Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos

#### Limpieza y preparación de columnas

#### Resolución